# Chapter 3. Dictionaries and Sets
---

## ToC


1. [Dictionary Views](#dictionary-views)  
2. [Practical Consequences of How dict Works](#practical-consequences-of-how-dict-works)

---

## Dictionary Views

In [1]:
d = dict(a=10, b=20, c=30)
d_values = d.values()
d_keys = d.keys()
d_items = d.items()

In [ ]:
print(f"Dict Keys: {d_keys}")
print(f"Dict Values: {d_values}")
print(f"Dict Items: {d_items}")

Dict Keys: dict_keys(['a', 'b', 'c'])
Dict Values: dict_values([10, 20, 30])
Dict Items: dict_items([('a', 10), ('b', 20), ('c', 30)])


In [16]:
len(d_values)

3

In [12]:
print(*[f"{i}: {v}" for i, v in enumerate(d)], sep='\n')

0: a
1: b
2: c


In [13]:
reversed(d_values)

In [17]:
d_values

dict_values([10, 20, 30])

A view object is a dynamic proxy. If the source dict is updated, you can immediately
see the changes through an existing view.

In [18]:
d['z'] = 99

In [19]:
d_values

dict_values([10, 20, 30, 99])

The classes `dict_keys`, `dict_values`, and `dict_items` are internal: they are not available
via `__builtins__` or any standard library module, and even if you get a reference
to one of them, you can’t use it to create a view from scratch in Python code:

In [20]:
values_class = type({}.values())


In [21]:
v = values_class()

TypeError: cannot create 'dict_values' instances

## Practical Consequences of How dict Works

The hash table implementation of Python's dict is very efficient, but it's important to
understand the practical effects of this design:

- Keys must be hashable objects. They must implement proper `__hash__` and
`__eq__` methods

- Item access by key is very fast. A `dict` may have millions of keys, but Python can
locate a key directly by computing the hash code of the key and deriving an index
offset into the hash table.

- Despite its new compact layout, dicts inevitably have a significant memory overhead. Python needs to keep at least one-third of the hash table rows empty to remain efficient.

- To save memory, avoid creating instance attributes outside of the `__init__` method.

### Compare memories 

That last tip about instance attributes comes from the fact that Python’s default
behavior is to store instance attributes in a special `__dict__` attribute, which is a dict
attached to each instance. 
Since [PEP 412—Key-Sharing Dictionary](https://peps.python.org/pep-0412/) was implemented
in Python 3.3, instances of a class can share a common hash table, stored with the
class. That common hash table is shared by the `__dict__` of each new instance that
has the same attributes names as the first instance of that class when `__init__`
returns. Each instance `__dict__` can then hold only its own attribute values as a simple
array of pointers. Adding an instance attribute after `__init__` forces Python to
create a new hash table just for the `__dict__` of that one instance (which was the
default behavior for all instances before Python 3.3). According to PEP 412, this optimization
reduces memory use by 10% to 20% for object-oriented programs

In [22]:
import sys

class WithDynamicAttributes:
    def __init__(self):
        self.a = 1

def create_with_dynamic():
    obj = WithDynamicAttributes()
    obj.b = 2  # added outside __init__
    return obj

class WithInitOnly:
    def __init__(self):
        self.a = 1
        self.b = 2  # added inside __init__

class WithSlots:
    __slots__ = ('a', 'b')
    
    def __init__(self):
        self.a = 1
        self.b = 2

def show_memory(obj, label):
    print(f"--- {label} ---")
    print(f"Object size: {sys.getsizeof(obj)} bytes")
    if hasattr(obj, '__dict__'):
        print(f"__dict__ size: {sys.getsizeof(obj.__dict__)} bytes")
        print(f"Attributes in __dict__: {obj.__dict__}")
    else:
        print("No __dict__ (uses __slots__ instead)")
    print()

# Create instances
obj_dynamic = create_with_dynamic()
obj_init = WithInitOnly()
obj_slots = WithSlots()

# Show memory usage
show_memory(obj_dynamic, "Dynamic Attribute (added outside __init__)")
show_memory(obj_init, "Init Only (attributes set in __init__)")
show_memory(obj_slots, "With Slots (memory optimized)")


--- Dynamic Attribute (added outside __init__) ---
Object size: 48 bytes
__dict__ size: 296 bytes
Attributes in __dict__: {'a': 1, 'b': 2}

--- Init Only (attributes set in __init__) ---
Object size: 48 bytes
__dict__ size: 296 bytes
Attributes in __dict__: {'a': 1, 'b': 2}

--- With Slots (memory optimized) ---
Object size: 48 bytes
No __dict__ (uses __slots__ instead)



More info on [Internals of sets and dicts](https://fpy.li/hashint)